section*{Project report - Integer Programming - Klaas Fiete Krutein}

subsection{Problem 1}
Code up Formulation 0, Formulation 2, and Formulation 3 for Gurobi or for CPLEX, using a language like C++ or Python. You should not hard-code any of the problem data; your codes should work for any values of $n$ and $c$. If you have to recompile your codes between instances, you are doing it wrong. Your code should read data from an external file.

The objective of this project is to assess the effect of different solver methodology on the time to find an optimal solution and the complexity of the algorithm. This report is presented as a combined code and report script. It is structured beginning with the import of the source data, the definition of the model formulation in the \texttt{pyomo} modeling interface package, the solving of instances using different parameter settings and finishing with a comparative analysis of the model outputs. 

\subsection*{Model formulations}

In [1]:
from pyomo.environ import *
import glob
import pandas as pd
import time
import numpy as np

We begin by modeling the assigment relaxation formulation. The data, parameter, variable and constraint formulations correspond to the formulation as given in the project description.

In [2]:
## Modeling framework assignment relaxation formulation (0)

def assignment_relax(dist):
    
    # Create model
    m = ConcreteModel()
    
    ## DATA
    
    # initialize number of points in problem
    m.P = Param(within=PositiveIntegers, initialize=len(dist)-1)
    # initialize indices for points in problem
    m.V = RangeSet(0,m.P)
    
    # Transform matrix to dictionary
    dist.to_dict()
    dist = dist.stack().to_dict()
    #print(dist)
    
    ## PARAMETERS
    
    # initialize cost parameters
    m.c = Param(m.V, m.V, initialize = dist)
    #m.c.pprint()
    
    ## VARIABLES
    
    # connection variable
    m.x = Var(m.V, m.V, within = Binary, initialize = 0)
    #m.x.pprint()
    
    ## CONSTRAINTS
    
    # Constraint 1b
    def single_connect1(model, i, j):
        return(sum(m.x[i,j] for j in m.V) == 1)
    m.s_con1 = Constraint(m.V, m.V, rule = single_connect1)
    #m.s_con1.pprint()
    
    # Constraint 1c
    def single_connect2(model, i, j):
        return(sum(m.x[i,j] for i in m.V) == 1)
    m.s_con2 = Constraint(m.V, m.V, rule = single_connect2)
    #m.s_con2.pprint()
    
    # Constraint 1d
    def no_self_routes(model, i):
        return(m.x[i,i] == 0)
    m.zeros = Constraint(m.V, rule = no_self_routes)
    #m.zeros.pprint()
    
    ## OBJECTIVE
    
    def objective_rule(m):
        return(sum(sum(m.c[i,j] * m.x[i,j] for j in m.V) for i in m.V))
    m.objective = Objective(rule = objective_rule, sense = minimize, doc='Define objective function')
    
    return(m)

We continue with the MTZ formulation. As for the assignment relaxation formulation, the definitions follow the definitions from the project description.

In [3]:
## Modeling framework MTZ formulation

# For testing purposes of random starting vertex
np.random.seed(123)

def MTZ(dist):
    
    # Create model
    m = ConcreteModel()
    
    ## DATA
    
    # initialize number of points in problem
    m.P = Param(within=PositiveIntegers, initialize=len(dist)-1)
    # initialize indices for points in problem
    m.V = RangeSet(0,m.P)
    
    # Transform matrix to dictionary
    dist.to_dict()
    dist = dist.stack().to_dict()
    #print(dist)
    
    ## PARAMETERS
    
    # initialize cost parameters
    m.c = Param(m.V, m.V, initialize = dist)
    #m.c.pprint()
    
    ## VARIABLES
    
    # connection variable
    m.x = Var(m.V, m.V, within = Binary, initialize = 0)
    #m.x.pprint()
    
    # tour position variable
    m.u = Var(m.V, within = NonNegativeReals, bounds = (0,(len(m.V)-1)), initialize = 0)
    #m.u.pprint()
    
    ## CONSTRAINTS
    
    # generate random starting vertex
    r = int(np.random.uniform(0,28,1))
    #print(r)
    
    # Constraint 1b
    def single_connect1(model, i, j):
        return(sum(m.x[i,j] for j in m.V) == 1)
    m.s_con1 = Constraint(m.V, m.V, rule = single_connect1)
    #m.s_con1.pprint()
    
    # Constraint 1c
    def single_connect2(model, i, j):
        return(sum(m.x[i,j] for i in m.V) == 1)
    m.s_con2 = Constraint(m.V, m.V, rule = single_connect2)
    #m.s_con2.pprint()
    
    # Constraint 1d
    def no_self_routes(model, i):
        return(m.x[i,i] == 0)
    m.zeros = Constraint(m.V, rule = no_self_routes)
    #m.zeros.pprint()
    
    # Constraint 3a    
    def order(model, i, j):
        if i == r or j == r:
            return(Constraint.Skip) 
        else:
            return(m.u[i] - m.u[j] + 1 <= (len(m.V)-1) * (1 - m.x[i,j]))
    m.order_con = Constraint(m.V, m.V, rule = order)
    #m.order_con.pprint()
    
    # Constraint 3b
    def initial_vertex(model, i):
        if i == r:
            return(m.u[i] == 0) # using python syntax we start indexing at position zero
        else:
            return(Constraint.Skip)
    m.start_at = Constraint(m.V, rule = initial_vertex)
    #m.start_at.pprint()
    
    # Constraint 3c 
    def later_vertex(model, i):
        if i == r:
            return(Constraint.Skip)
        else:
            return(inequality(1, m.u[i], len(m.V)-1)) # again we need to consider the indexing change
    m.continue_at = Constraint(m.V, rule = later_vertex)
    #m.continue_at.pprint()

    ## OBJECTIVE
    
    def objective_rule(m):
        return(sum(sum(m.c[i,j] * m.x[i,j] for j in m.V) for i in m.V))
    m.objective = Objective(rule = objective_rule, sense = minimize, doc='Define objective function')
    
    return(m)

Lastly, we create the function for the MCF formulation, which also follows the same description as the project task.

In [4]:
## Modeling framework MCF formulation (3)

# For testing purposes of random starting vertex
np.random.seed(123)

def MCF(dist):
    
    # Create model
    m = ConcreteModel()
    
    ## DATA
    
    # initialize number of points in problem
    m.P = Param(within=PositiveIntegers, initialize=len(dist)-1)
    # initialize indices for points in problem
    m.V = RangeSet(0,m.P)
    #m.V.pprint()
    
    # Transform matrix to dictionary
    dist.to_dict()
    dist = dist.stack().to_dict()
    #print(dist)
    
    ## PARAMETERS
    
    # initialize cost parameters
    m.c = Param(m.V, m.V, initialize = dist)
    #m.c.pprint()
    
    ## VARIABLES
    
    # connection variable
    m.x = Var(m.V, m.V, within = Binary, initialize = 0)
    #m.x.pprint()
    
    # flow variable
    m.f = Var(m.V, m.V, m.V, within = NonNegativeReals, bounds = (0, 1), initialize = 0)
    
    ## CONSTRAINTS
    
    # generate random starting vertex
    r = int(np.random.uniform(0,28,1))
    #print(r)
    
    # generate a subset of vertices that excludes r
    m.sub_V = Set(dimen = 1)
    for k in m.V:
        if k != r:
            m.sub_V.add(k)
    #m.sub_V.pprint()
            
    # Constraint 1b
    def single_connect1(model, i, j):
        return(sum(m.x[i,j] for j in m.V) == 1)
    m.s_con1 = Constraint(m.V, m.V, rule = single_connect1)
    #m.s_con1.pprint()
    
    # Constraint 1c
    def single_connect2(model, i, j):
        return(sum(m.x[i,j] for i in m.V) == 1)
    m.s_con2 = Constraint(m.V, m.V, rule = single_connect2)
    #m.s_con2.pprint()
    
    # Constraint 1d
    def no_self_routes(model, i):
        return(m.x[i,i] == 0)
    m.zeros = Constraint(m.V, rule = no_self_routes)
    #m.zeros.pprint()
    
    # Constraint 4a
    def flows_from_r(model, v):
        if v == r:
            return(Constraint.Skip)
        else:
            return(sum(m.f[v,r,j] for j in m.sub_V) - sum(m.f[v,j,r] for j in m.sub_V) == 1)
    m.out_flows = Constraint(m.V, rule = flows_from_r)
    #m.out_flows.pprint()
    
    # Constraint 4b
    def flow_conservation(model, i, v):
        if i == r or v == r or i == v:
            return(Constraint.Skip)
        else:
            # generate a subset of vertices that excludes i
            m.sub_V1 = Set(dimen = 1)
            for k in m.V:
                if k != i:
                    m.sub_V1.add(k)
            #m.sub_V1.pprint()
            constr = (sum(m.f[v,i,j] for j in m.sub_V1) - sum(m.f[v,j,i] for j in m.sub_V1) == 0)
            m.del_component(m.sub_V1)
            return(constr)
    m.flow_con = Constraint(m.V, m.V, rule = flow_conservation)
    #m.flow_con.pprint()
            
    # Constraint 4c
    def link(model, i, j, v):
        if v == r:
            return(Constraint.Skip)
        else:
            return(m.f[v,i,j] <= m.x[i,j])
    m.link_con = Constraint(m.V, m.V, m.V, rule = link) 
    
    ## OBJECTIVE
    
    def objective_rule(m):
        return(sum(sum(m.c[i,j] * m.x[i,j] for j in m.V) for i in m.V))
    m.objective = Objective(rule = objective_rule, sense = minimize, doc='Define objective function')
    
    return(m)

\subsection*{Data}
Having defined the model formulations, we can continue with reading in the data and creating model formulation instances for each formulation for each testing dataset. We begin by reading in the data. 

In [5]:
# read in source files for test data
source = r'/Users/fietekrutein/Documents/University/University of Washington/Courses/2020 Q1/IND E 599 Integer Programming/Project' # use your path
all_files = glob.glob(source + "/*.txt")

# test datasets
bays29 = pd.read_csv(source + '/bays29.txt', delimiter = "\t", skiprows = 1, header = None)
dantzig42 = pd.read_csv(source + '/dantzig42.txt', delimiter = "\t", skiprows = 1, header = None)
pr76 = pd.read_csv(source + '/pr76.txt', delimiter = "\t", skiprows = 1, header = None)
rat99 = pd.read_csv(source + '/rat99.txt', delimiter = "\t", skiprows = 1, header = None)

We now generate instances of the assignment relaxation formulation for each dataset.

In [6]:
# create assignment relaxation formulations for each dataset
bays29_assignment_relax_m = assignment_relax(bays29)
dantzig42_assignment_relax_m = assignment_relax(dantzig42)
pr76_assignment_relax_m = assignment_relax(pr76)
rat99_assignment_relax_m = assignment_relax(rat99)

We further generate instances for the MTZ formulation.

In [7]:
# create MTZ formulations for each dataset
bays29_MTZ_m = MTZ(bays29)
dantzig42_MTZ_m = MTZ(dantzig42)
pr76_MTZ_m = MTZ(pr76)
rat99_MTZ_m = MTZ(rat99)

Lastly, we create model instances for the MCF formulation.

In [8]:
# create MCF formulations for each dataset
bays29_MCF_m = MCF(bays29)
dantzig42_MCF_m = MCF(dantzig42)
pr76_MCF_m = MCF(pr76)
rat99_MCF_m = MCF(rat99)

\subsection*{Solver Definition}
We can now proceed to the solver definition. It was decided to create a single solver function that once called iterates the solving process for the passed instance and changing the parameter settings on \textit{Presolve} and \textit{Cuts} on the fly. The function therefore returns four results:
\begin{enumerate}
\item Default settings
\item No pre-solve
\item No cuts
\item No pre-solve and no cuts
\end{enumerate}
We further pass it an indicator whether to only solve the default solver, since we will not need the alternative solver configurations for the assignment relaxation formulation. 

In [9]:
# Define solving instance
from gurobipy import *

timelimit = 3600 # in seconds

def solve(m, default):
    
    ## SOLVE

    if __name__ == '__main__':
        from pyomo.opt import SolverFactory
        import pyomo.environ
        
        # Create instances for each model setting
        m_default = m
        if default == False:
            m_no_presolve = m
            m_no_cuts = m
            m_no_presolve_no_cuts = m
    
        opt = SolverFactory('gurobi')
        opt.options['IntFeasTol']= 10e-10
        opt.options['MIPGap'] = 0
        opt.options['TimeLimit'] = timelimit
        
        # Solve default
        print('')
        print('DEFAULT')
        print('')
        results_default = opt.solve(m_default, load_solutions=False, tee = True)
        print(results_default)
        
        
        if default == False:
            # Solve without presolve if enabled
            print('')
            print('NO PRESOLVE')
            print('')
            opt.options['Presolve'] = 0 # disable presolve
            results_no_presolve = opt.solve(m_no_presolve, load_solutions=False, tee = True)
            print(results_no_presolve)

        
            # Solve without cuts if enabled
            print('')
            print('NO CUTS')
            print('')
            opt.options['Presolve'] = -1 
            opt.options['Cuts'] = 0 # disable cuts
            results_no_cuts = opt.solve(m_no_cuts, load_solutions=False, tee = True)
            print(results_no_cuts)
        
        
            # Solve without cuts and without presolve if enabled
            print('')
            print('NO PRESOLVE NO CUTS')
            print('')
            opt.options['Presolve'] = 0 # disable presolve
            opt.options['Cuts'] = 0 # disable cuts
            results_no_presolve_no_cuts = opt.solve(m_no_presolve_no_cuts, load_solutions=False, tee = True)
            print(results_no_presolve_no_cuts)

\subsection{Solutions}
We continue by solving each model instance using the solver function defined above. Since the model output is quite large in size and would not fit the format of a report very well, we comment out the solver command in this script and report the model results in table format below. We start with the assignment relaxation formulation.

In [10]:
## Assignment relaxation formulation
# solve each model instances
#print('Assignment relaxation formulation')
#print('Bays29 Dataset:')
#solve(bays29_assignment_relax_m, default = True)

In [11]:
#print('Assignment relaxation formulation')
#print('Dantzig42 Dataset:')
#solve(dantzig42_assignment_relax_m, default = True)

In [12]:
#print('Assignment relaxation formulation')
#print('Pr76 Dataset:')
#solve(pr76_assignment_relax_m, default = True)

In [13]:
#print('Assignment relaxation formulation')
#print('Rat99 Dataset:')
#solve(rat99_assignment_relax_m, default = True)

We obtain the results as presented in Table 1 from the assignment relaxation formulation.
\begin{tabular}{l*{6}{c}r}
Team              & P & W & D & L & F  & A & Pts \\
\hline
Manchester United & 6 & 4 & 0 & 2 & 10 & 5 & 12  \\
Celtic            & 6 & 3 & 0 & 3 &  8 & 9 &  9  \\
Benfica           & 6 & 2 & 1 & 3 &  7 & 8 &  7  \\
FC Copenhagen     & 6 & 2 & 1 & 3 &  5 & 8 &  7  \\
\end{tabular}

In [14]:
## MTZ formulation
# solve each model instances
#print('MTZ formulation')
#print('Bays29 Dataset:')
#solve(bays29_MTZ_m, default = False)

In [15]:
#print('MTZ formulation')
#print('Dantzig42 Dataset:')
#solve(dantzig42_MTZ_m, default = False)

In [16]:
#print('MTZ formulation')
#print('Pr76 Dataset:')
solve(pr76_MTZ_m, default = False)


DEFAULT

Using license file /Users/fietekrutein/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/3f/ty6j5m0x59l2j7rp1ktlmbx80000gn/T/tmpl_q36kpn.pyomo.lp
Reading time = 0.54 seconds
x5853: 17405 rows, 5853 columns, 894905 nonzeros
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Changed value of parameter MIPGap to 0.0
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 17405 rows, 5853 columns and 894905 nonzeros
Model fingerprint: 0x0cbff3fe
Variable types: 77 continuous, 5776 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 8e+01]
  RHS range        [1e+00, 8e+01]
Presolve removed 11703 rows and 78 columns
Presolve time:

 29489 14480 108591.856   85  192 111137.000 105095.650  5.44%  22.7  360s
 29495 14484 109801.222   78  192 111137.000 105095.650  5.44%  22.7  365s
 29500 14487 109886.735   86  207 111137.000 105095.650  5.44%  22.7  370s
 29506 14491 107117.847   60  202 111137.000 105095.650  5.44%  22.7  375s
 29511 14494 108894.076   52  189 111137.000 105095.650  5.44%  22.7  380s
 29517 14498 106271.750   37  208 111137.000 105095.650  5.44%  22.7  385s
 29522 14502 106810.351   65  198 111137.000 105095.650  5.44%  22.7  390s
 29528 14506 108510.083  200  195 111137.000 105095.650  5.44%  22.6  395s
 29534 14510 107497.093   55  197 111137.000 105095.650  5.44%  22.6  400s
 29540 14514 107491.432  133  197 111137.000 105095.650  5.44%  22.6  405s
 29545 14517 105840.230   72  190 111137.000 105095.650  5.44%  22.6  410s
 29550 14520 108917.160   79  194 111137.000 105095.650  5.44%  22.6  415s
 29554 14523 108321.863   85  196 111137.000 105095.650  5.44%  22.6  420s
 29555 14524 109809.878  

 76009 30175     cutoff   90      109200.000 105746.653  3.16%  22.3  921s
 76318 30175 105746.653   85   84 109200.000 105746.653  3.16%  22.4  928s
 76493 30312 107168.189  139   49 109200.000 105746.653  3.16%  22.4  931s
 76948 30449 106245.443  109   57 109200.000 105746.653  3.16%  22.5  936s
 77259 30570 106531.376   81  147 109200.000 105746.653  3.16%  22.5  941s
 77647 30686 infeasible  117      109200.000 105746.653  3.16%  22.6  945s
 78312 30959 106495.500   98   92 109200.000 105746.653  3.16%  22.8  951s
 78890 31198 106402.946   78   95 109200.000 105746.653  3.16%  22.9  956s
 79586 31450 105791.788   87  197 109200.000 105746.653  3.16%  22.9  961s
 80314 31808 107532.032  114   63 109200.000 105746.653  3.16%  23.0  967s
 81153 32061 105987.286   96   87 109200.000 105746.653  3.16%  23.1  972s
 81500 32194 106419.885  119   36 109200.000 105746.653  3.16%  23.2  975s
 82513 32646 107572.262  109  128 109200.000 105746.653  3.16%  23.3  981s
 83025 32946 106867.583  

 117464 37719 106351.500  108   55 108234.000 105836.596  2.22%  27.3 1482s
 118190 37795 106827.460  164   88 108234.000 105843.926  2.21%  27.3 1486s
 118733 37978 106958.445  142  113 108234.000 105855.732  2.20%  27.4 1490s
 119319 37807 106881.500  121   74 108234.000 105860.331  2.19%  27.4 1497s
 119347 38197 107138.470  132  125 108234.000 105863.585  2.19%  27.4 1500s
 119909 38093 106760.117  166  134 108234.000 105870.113  2.18%  27.5 1530s
 120332 38507 107689.696  150  126 108234.000 105883.098  2.17%  27.5 1536s
 120894 38669     cutoff  103      108234.000 105893.605  2.16%  27.6 1540s
 122025 38989 107988.797  109   89 108234.000 105914.892  2.14%  27.7 1546s
 122972 39302 107761.033  109   53 108234.000 105929.429  2.13%  27.8 1552s
 123381 39481 107228.750  164  128 108234.000 105937.544  2.12%  27.8 1556s
 124468 39830 107615.000  151   34 108234.000 105950.500  2.11%  27.9 1563s
 125122 39947     cutoff  136      108234.000 105953.286  2.11%  28.0 1567s
 125713 4012

 198049 53332 106952.620  112  116 108159.000 106500.248  1.53%  33.8 2044s
 198574 53416 106885.866  115  182 108159.000 106502.634  1.53%  33.8 2049s
 199115 53525 107686.275  116   34 108159.000 106504.611  1.53%  33.8 2053s
 199692 53508 107710.223  169   70 108159.000 106507.259  1.53%  33.9 2058s
 200142 53569 107537.437  119   89 108159.000 106510.341  1.52%  33.9 2062s
 200678 53690 106729.000  133  125 108159.000 106512.583  1.52%  33.9 2065s
 201800 53847 infeasible  128      108159.000 106516.598  1.52%  34.0 2074s
 202300 53937 107672.918  119   55 108159.000 106518.866  1.52%  34.0 2078s
 202768 54243 infeasible  107      108159.000 106520.412  1.51%  34.0 2083s
 203316 54572 107948.870  103  106 108159.000 106521.247  1.51%  34.0 2088s
 203903 54842 107088.033  124   96 108159.000 106524.239  1.51%  34.0 2092s
 204470 55156     cutoff  153      108159.000 106526.433  1.51%  34.1 2097s
 205019 55341 106759.451  102   67 108159.000 106528.619  1.51%  34.1 2101s
 205498 5555

...


Problem: 
- Name: x5853
  Lower bound: 97407.0
  Upper bound: 110314.0
  Number of objectives: 1
  Number of constraints: 17405
  Number of variables: 5853
  Number of binary variables: 5776
  Number of integer variables: 5776
  Number of continuous variables: 77
  Number of nonzeros: 894905
  Sense: minimize
Solver: 
- Status: aborted
  Return code: 0
  Message: Optimization terminated because the time expended exceeded the value specified in the TimeLimit parameter.
  Termination condition: maxTimeLimit
  Termination message: Optimization terminated because the time expended exceeded the value specified in the TimeLimit parameter.
  Wall time: 3600.107353925705
  Error rc: 0
  Time: 3608.4455420970917
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: stoppedByLimit
  Message: Optimization terminated because the time expended exceeded the value specified in the TimeLimit parameter.
  Objective:
    __default_objective__:
      Value: 110314
 

   816   698 92507.2667   28  120 118962.000 80916.7867  32.0%  12.1   30s
H  836   677                    116768.00000 80916.7867  30.7%  12.1   31s
   942   739     cutoff   62      116768.000 80916.7867  30.7%  12.3   35s
  1122   830 103887.484   55   76 116768.000 80916.7867  30.7%  12.6   40s
  1300   885 114430.084  105   80 116768.000 80916.7867  30.7%  13.0   45s
  1666  1065 115458.933  154   72 116768.000 81357.3600  30.3%  12.3   50s
H 1827  1050                    115268.00000 81359.4667  29.4%  12.0   52s
H 1828   937                    112826.00000 81359.4667  27.9%  12.0   52s
  1963  1059 110982.813   95   64 112826.000 81359.4667  27.9%  12.1   55s
H 2074  1037                    112018.00000 81427.4133  27.3%  11.9   57s
  2234  1134 105293.053   37   75 112018.000 82020.0000  26.8%  11.8   60s
  2654  1399 105021.849   50   77 112018.000 82554.1867  26.3%  11.6   68s
H 2701  1396                    111921.00000 82554.1867  26.2%  11.6   68s
  2758  1638 110772.218  

 34492 22194 95453.0400   24  116 108159.000 86511.6533  20.0%  11.6  516s
 34884 22458 104093.723  112   85 108159.000 86530.0867  20.0%  11.7  521s
 35195 22579 101735.600   31   82 108159.000 86573.0133  20.0%  11.7  525s
 35547 22841 102460.244   43   81 108159.000 86580.4978  20.0%  11.7  531s
 35822 22941 104312.149   35   93 108159.000 86584.7900  19.9%  11.8  535s
 36117 23165 103229.761   45   80 108159.000 86616.7867  19.9%  11.8  540s
 36483 23311 infeasible   40      108159.000 86671.7600  19.9%  11.8  545s
 36679 23447 92571.9867   24   99 108159.000 86683.5200  19.9%  11.9  550s
 37166 23857 94979.2467   29  108 108159.000 86733.4711  19.8%  11.9  557s
 37505 23931 infeasible   77      108159.000 86748.0178  19.8%  11.9  561s
 37795 24216 97269.4967   33   94 108159.000 86751.3733  19.8%  11.9  566s
 38230 24445 infeasible   42      108159.000 86763.7200  19.8%  12.0  572s
 38375 24482 89567.6100   20  128 108159.000 86772.8000  19.8%  12.0  575s
 38493 24641 104640.807  

 69165 42648 106177.767   46   89 108159.000 88634.6133  18.1%  13.7 1067s
 69652 42935 infeasible   34      108159.000 88652.5333  18.0%  13.7 1073s
 70031 43007 infeasible   50      108159.000 88667.3267  18.0%  13.7 1075s
 70323 43211 90145.9867   20   98 108159.000 88673.9067  18.0%  13.7 1082s
 70508 43306     cutoff   62      108159.000 88674.8356  18.0%  13.7 1085s
 70924 43540 89614.1000   20  115 108159.000 88697.5511  18.0%  13.7 1092s
 71103 43712 90084.2511   20  129 108159.000 88702.0667  18.0%  13.7 1096s
 71471 43966     cutoff   49      108159.000 88714.8067  18.0%  13.7 1101s
 71952 44244 infeasible   58      108159.000 88730.2000  18.0%  13.7 1107s
 72187 44383 98034.0267   36   85 108159.000 88750.2800  17.9%  13.7 1111s
 72569 44716 105507.567   38   88 108159.000 88761.9267  17.9%  13.7 1117s
 72959 44791 101017.660   51   71 108159.000 88770.5556  17.9%  13.7 1120s
 73226 44994 100989.904  103   68 108159.000 88777.8800  17.9%  13.7 1126s
 73482 45166 104409.276  


Explored 105104 nodes (1491815 simplex iterations) in 5005.21 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 108159 108356 109176 ... 112018

Time limit reached
Best objective 1.081590000000e+05, best bound 8.972900000000e+04, gap 17.0397%

Problem: 
- Name: x5853
  Lower bound: 89729.0
  Upper bound: 108159.0
  Number of objectives: 1
  Number of constraints: 17405
  Number of variables: 5853
  Number of binary variables: 5776
  Number of integer variables: 5776
  Number of continuous variables: 77
  Number of nonzeros: 894905
  Sense: minimize
Solver: 
- Status: aborted
  Return code: 0
  Message: Optimization terminated because the time expended exceeded the value specified in the TimeLimit parameter.
  Termination condition: maxTimeLimit
  Termination message: Optimization terminated because the time expended exceeded the value specified in the TimeLimit parameter.
  Wall time: 5005.529363155365
  Error rc: 0
  Time: 5014.215648889542
Solution: 
- numbe

In [17]:
#print('MTZ formulation')
#print('Rat99 Dataset:')
#solve(rat99_MTZ_m, default = False)

In [18]:
## MCF formulation
# solve each model instances
#print('MCF formulation')
#print('Bays29 Dataset:')
#solve(bays29_MCF_m, default = False)

In [19]:
#print('MCF formulation')
#print('Dantzig42 Dataset:')
#solve(dantzig42_MCF_m, default = False)

In [20]:
#print('MCF formulation')
#print('Pr76 Dataset:')
#solve(pr76_MCF_m, default = False)

In [21]:
#print('MCF formulation')
#print('Rat99 Dataset:')
#solve(rat99_MCF_m, default = False)